 **Problem 3**


https://colab.research.google.com/drive/1aSECHvD_jfC-6ErJMXUFT6RMJFiYyUfJ


You must use the data generation script: https://github.com/jonPlante/IFT6135_Ass1/blob/master/generateData.py
to prepare the data

Then save on your drive as catDog.pkl.gz

In [ ]:
!pip3 install 'torch==0.4.0'
!pip3 install 'torchvision==0.2.1'
!pip3 install --no-cache-dir -I 'pillow==5.1.0'

# Restart Kernel
# This workaround is needed to properly upgrade PIL on Google Colab.
import os
os._exit(00)

In [ ]:
!pip install git+https://github.com/aleju/imgaug
from imgaug import augmenters as iaa
import imgaug as ia

In [ ]:
import numpy as np
import gzip
import pickle
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
with gzip.open('/content/gdrive/My Drive/catDog.pkl.gz', 'rb') as f:
  train_set, valid_set, test_set = pickle.load(f, encoding='latin1')


In [ ]:
import torch
import PIL
from PIL import Image
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset


In [ ]:
torch_transform = torchvision.transforms.Compose([
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(20),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.RandomGrayscale(),
    torchvision.transforms.RandomAffine(20, translate=[0.1,0.1], scale=None, shear=5, resample=False, fillcolor=0),
    torchvision.transforms.RandomResizedCrop(64, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
    torchvision.transforms.ToTensor(),
])

In [ ]:
class ImgAugTransform:
  def __init__(self):
    self.aug = iaa.Sequential([
        iaa.Scale((64, 64)),
        iaa.Sometimes(0.6, iaa.GaussianBlur(sigma=(0, 2.0))),
        iaa.Fliplr(0.5),
        iaa.Affine(rotate=(-20, 20), mode='symmetric'),
        iaa.Sometimes(0.6,
                      iaa.OneOf([iaa.Dropout(p=(0, 0.1)),
                                 iaa.CoarseDropout(0.1, size_percent=0.7)])),
        iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)
    ])
      
  def __call__(self, img):
    img = np.array(img)
    return self.aug.augment_image(img)

Img_Aug_T = ImgAugTransform()

In [ ]:
import random
random.shuffle(train_set)
train=[]
valid=[]
test=[]
for i in range(len(train_set)):
  x=torch.Tensor(train_set[i][0])
  train.append([x,train_set[i][1]])
  #data augmentation
  for j in range(5):
    #pytorch
    x=torch_transform(Image.fromarray(np.uint8(np.moveaxis(train_set[i][0],0,2)*255)))
    train.append([x,train_set[i][1]])
    #dataAugment
    x=Img_Aug_T(np.uint8(np.moveaxis(train_set[i][0],0,2)*255))
    x=torch.Tensor(np.moveaxis(x/255,2,0))
    train.append([x,train_set[i][1]])
  train_set[i]=[]
  if i%500==0 and not i==0:
    print('\t'+str(np.round(i/len(train_set)*100,2))+'% complete')
train_set=[]
for i in range(len(valid_set)):
  x=torch.Tensor(valid_set[i][0])
  valid.append([x,valid_set[i][1]])
  valid_set[i]=[]
valid_set=[]
for i in range(len(test_set)):
  x=torch.Tensor(test_set[i][0])
  test.append([x,test_set[i][1]])
  test_set[i]=[]
test_set=[]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3,padding=1)
        
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1) 
        self.conv3 = nn.Conv2d(64, 128, 3,padding=1,stride=2)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.conv5 = nn.Conv2d(64, 128, 1,padding=0,stride=2)
        self.conv6 = nn.Conv2d(128, 128, 3,padding=1)
        self.conv7 = nn.Conv2d(128, 256, 3,padding=1,stride=2)
        self.conv8 = nn.Conv2d(256, 256, 3,padding=1)
        self.conv9 = nn.Conv2d(128, 256, 1,padding=0,stride=2)
        self.conv10 = nn.Conv2d(256, 512, 3,padding=1,stride=2)
        self.conv11 = nn.Conv2d(512, 512, 3,padding=1)
        self.conv12 = nn.Conv2d(256, 512, 1,padding=0,stride=2)
        
        self.fc1 = nn.Linear(512*1*1,1000)
        self.fc2 = nn.Linear(1000, 2)
        self.AvgPool = nn.AvgPool2d(8,1)
       

    def forward(self, x):
        x = F.relu(self.conv1(x))
        residual_1=x
        
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv2(x)+residual_1)
        residual_2=x
        
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv2(x)+residual_2)
        residual_3=x
        
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv2(x)+residual_3)
        residual_4=x
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x)+self.conv5(residual_4))
        residual_5=x
        
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv6(x)+residual_5)
        residual_6=x
        
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv6(x)+residual_6)
        residual_7=x
        
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv6(x)+residual_7)
        residual_8=x
        
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x)+self.conv9(residual_8))
        residual_9=x
        
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv8(x)+residual_9)
        residual_10=x
        
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv8(x)+residual_10)
        residual_11=x
        
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv8(x)+residual_11)
        residual_12=x
        
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv8(x)+residual_12)
        residual_13=x
        
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv8(x)+residual_13)
        residual_14=x
        
        x = F.relu(self.conv10(x))
        x = F.relu(self.conv11(x)+self.conv12(residual_14))
        residual_15=x
        
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv11(x)+residual_15)
        residual_16=x
        
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv11(x)+residual_16)
        x=self.AvgPool(x)

        x = x.view(-1, 512*1*1)
        x = F.relu(self.fc1(x))

        x = self.fc2(x)

                
        return x



In [ ]:
epochs=60
start=31
MB=50
LR=0.001
use_cuda = True

numTrain=len(train)
numValid=len(valid)




net = Net()
#if want to load modelcomment
#net.load_state_dict(torch.load('/content/gdrive/My Drive/model_'+str(start)+'.pkl'))


if use_cuda and torch.cuda.is_available():
    net.cuda()
    print('using cuda')
else:
  print('not using cuda')

trainloader = torch.utils.data.DataLoader(train, batch_size=MB,shuffle=False, num_workers=2)
validloader = torch.utils.data.DataLoader(valid, batch_size=MB,shuffle=False, num_workers=2)


criterion = nn.CrossEntropyLoss()#nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.0)

train_data=np.zeros((epochs+1,3))
valid_data=np.zeros((epochs+1,3))

print('Calculating initial training loss')
total_loss = 0.0
total_error=0
for i, data in enumerate(trainloader, 0):
 # get the inputs
  images, labels = data
  images=Variable(images)
  labels=Variable(labels)#.type(torch.FloatTensor))
 
  if use_cuda and torch.cuda.is_available():
    images = images.cuda()
    labels = labels.cuda()
 
  outputs = net(images)
  loss = criterion(outputs, labels)
  total_error+=np.sum(np.abs(np.argmax(outputs.data.cpu().numpy(),axis=1)-labels.cpu().numpy()))
 
  total_loss+=loss.data.cpu()
  if i*MB%10000==0 and not i==0:
    print('\t'+str(np.round(i*MB/numTrain*100,2))+'% complete')

   
train_data[0,:]=[0,total_loss/(numTrain/MB),total_error/numTrain]
print('Initial training loss: ' +str(np.round(train_data[0,1],2))+', training error: '+str(np.round(train_data[0,2]*100,2))+'%, training accuracy:'+str(np.round((1-train_data[0,2])*100,2))+'%')


print('Calculating initial validation loss')
total_loss = 0.0
total_error=0
for i, data in enumerate(validloader, 0):
  # get the inputs
  images, labels = data
  images=Variable(images)
  labels=Variable(labels)#.type(torch.FloatTensor))
  
  if use_cuda and torch.cuda.is_available():
    images = images.cuda()
    labels = labels.cuda()
  
  outputs = net(images)
  loss = criterion(outputs, labels)
  total_error+=np.sum(np.abs(np.argmax(outputs.data.cpu().numpy(),axis=1)-labels.cpu().numpy()))
  total_loss+=loss.data.cpu()
    
valid_data[0,:]=[0,total_loss/(numValid/MB),total_error/numValid]
print('Initial validation loss: ' +str(np.round(valid_data[0,1],2))+', validation error: '+str(np.round(valid_data[0,2]*100,2))+'%, validation accuracy:'+str(np.round((1-valid_data[0,2])*100,2))+'%')


for epoch in range(start,epochs):  # loop over the dataset multiple times
  
  trainloader=[]
  validloader=[]
  train=[]
  valid=[]
  test=[]
  
  with gzip.open('/content/gdrive/My Drive/catDog.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
  
  random.shuffle(train_set)
  
  for i in range(len(train_set)):
    x=torch.Tensor(train_set[i][0])
    train.append([x,train_set[i][1]])
     #data augmentation
    for j in range(3):
      x=torch_transform(Image.fromarray(np.uint8(np.moveaxis(train_set[i][0],0,2)*255)))
      train.append([x,train_set[i][1]])
      x=Img_Aug_T(np.uint8(np.moveaxis(train_set[i][0],0,2)*255))
      x=torch.Tensor(np.moveaxis(x/255,2,0))
      train.append([x,train_set[i][1]])
    train_set[i]=[]
    if i%500==0 and not i==0:
      print('\t'+str(np.round(i/len(train_set)*100,2))+'% complete')
  train_set=[]
  for i in range(len(valid_set)):
    x=torch.Tensor(valid_set[i][0])
    valid.append([x,valid_set[i][1]])
    valid_set[i]=[]
  valid_set=[]
  for i in range(len(test_set)):
    x=torch.Tensor(test_set[i][0])
    test.append([x,test_set[i][1]])
    test_set[i]=[]
  test_set=[]

  trainloader = torch.utils.data.DataLoader(train, batch_size=MB,shuffle=False, num_workers=2)
  validloader = torch.utils.data.DataLoader(valid, batch_size=MB,shuffle=False, num_workers=2)
  
  running_loss = 0.0
  num_ex=0
  for i, data in enumerate(trainloader, 0):
    # get the inputs
    images, labels = data
    images=Variable(images)
    labels=Variable(labels)#.type(torch.FloatTensor))
  
    if use_cuda and torch.cuda.is_available():
      images = images.cuda()
      labels = labels.cuda()
        
    # zero the parameter gradients
    optimizer.zero_grad()
    
    # forward + backward + optimize
    outputs = net(images)
    #print(outputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.data.cpu().numpy()
    num_ex+=1
    if i*MB%10000 == 0 and not i==0:
      print('\tLoss: '+str(np.round(running_loss/num_ex,3))+', '+str(np.round(i*MB/numTrain*100,2))+'% of epoch ' + str(epoch+1) +' complete')
      num_ex=0
      running_loss=0
  print('Epoch '+str(epoch+1) + ' complete')
  
  torch.save(net.state_dict(), '/content/gdrive/My Drive/model_'+str(epoch+1)+'.pkl')
                    
  print('Calculating training loss')
  total_loss = 0.0
  total_error=0
  for i, data in enumerate(trainloader, 0):
    # get the inputs
    images, labels = data
    images=Variable(images)
    labels=Variable(labels)#.type(torch.FloatTensor))
    
    if use_cuda and torch.cuda.is_available():
      images = images.cuda()
      labels = labels.cuda()
  
    outputs = net(images)
    loss = criterion(outputs, labels)
    total_error+=np.sum(np.abs(np.argmax(outputs.data.cpu().numpy(),axis=1)-labels.cpu().numpy()))
    total_loss+=loss.data.cpu()
    if i*MB%10000==0 and not i==0:
      print('\t'+str(np.round(i*MB/numTrain*100,2))+'% complete')
    
  train_data[epoch+1,:]=[epoch+1,total_loss/(numTrain/MB),total_error/numTrain]
  print('Training loss: ' +str(np.round(train_data[epoch+1,1],2))+', training error: '+str(np.round(train_data[epoch+1,2]*100,2))+'%, training accuracy:'+str(np.round((1-train_data[epoch+1,2])*100,2))+'%')

  print('Calculating validation loss')
  total_loss = 0.0
  total_error=0
  for i, data in enumerate(validloader, 0):
    # get the inputs
    images, labels = data
    images=Variable(images)
    labels=Variable(labels)#.type(torch.FloatTensor))
  
    if use_cuda and torch.cuda.is_available():
      images = images.cuda()
      labels = labels.cuda()
  
    outputs = net(images)
    loss = criterion(outputs, labels)
    total_error+=np.sum(np.abs(np.argmax(outputs.data.cpu().numpy(),axis=1)-labels.cpu().numpy()))
    total_loss+=loss.data.cpu()
    
  valid_data[epoch+1,:]=[epoch+1,total_loss/(numValid/MB),total_error/numValid]
  print('Validation loss: ' +str(np.round(valid_data[epoch+1,1],2))+', validation error: '+str(np.round(valid_data[epoch+1,2]*100,2))+'%, validation accuracy:'+str(np.round((1-valid_data[epoch+1,2])*100,2))+'%')
         
print('Finished Training')
torch.cuda.empty_cache()
            


In [ ]:
# if want to load model uncomment

#net = Net()
#net.load_state_dict(torch.load('/content/gdrive/My Drive/model_'+str(31)+'.pkl'))
#use_cuda=True
#
#if use_cuda and torch.cuda.is_available():
#    net.cuda()
#    print('using cuda')
#else:
#  print('not using cuda')
#

print('Calculating performance on test set')
testloader = torch.utils.data.DataLoader(test, batch_size=1,shuffle=False, num_workers=2)
test_data=np.zeros((len(test),2))
for i, data in enumerate(testloader, 0):
# get the inputs
  images, labels = data
  images=Variable(images)
  labels=Variable(labels[0])
  
  if use_cuda and torch.cuda.is_available():
    images = images.cuda()
    labels = labels.cuda()
  
  outputs = net(images)
  #print(labels.cpu().numpy())
  test_data[i,:]=[labels.cpu().numpy(),np.argmax(outputs.data.cpu().numpy())]
test_data=test_data[np.argsort(test_data[:, 0])]

print('done')


In [ ]:
import csv
with open('/content/gdrive/My Drive/submission_file.csv','w') as csvFile:
    writer = csv.writer(csvFile,lineterminator = '\n')
    writer.writerow(['id','label'])
    for i in range(test_data.shape[0]):
      if test_data[i,1]==0:
        writer.writerow([test_data[i,0].astype(np.int32),'Cat'])
      else:
        writer.writerow([test_data[i,0].astype(np.int32),'Dog'])